In [8]:
import pandas as pd

# Load your Excel file
data = pd.read_csv(r"D:\Graduation Project\data\hand_gesture_project\data\augmented_sensor_data_300k.csv")

In [9]:
# Clean column names
data.columns = data.columns.str.strip().str.lower().str.replace(" ", "_")

# Drop missing values
data_cleaned = data.dropna()

# Define numerical features
features = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']


In [10]:
# Extract features and labels
features = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
X_raw = data[features]
y_raw = data['lable']


# Recreate the DataFrame with labels
normalized_data = pd.DataFrame(X_raw, columns=features)
normalized_data['lable'] = y_raw.values

In [11]:
import numpy as np

def create_sequences(data, features, window_size=100, stride=50):
    sequences = []
    labels = []
    for label in data['lable'].unique():
        class_data = data[data['lable'] == label]
        class_values = class_data[features].values
        for start in range(0, len(class_values) - window_size + 1, stride):
            end = start + window_size
            seq = class_values[start:end]
            sequences.append(seq)
            labels.append(label)
    return np.array(sequences), np.array(labels)


In [12]:
# Apply function
X_seq, y_seq = create_sequences(normalized_data, features)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_seq)
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [13]:
# Shuffle the data
X_seq, y_encoded = shuffle(X_seq, y_encoded, random_state=42)

In [15]:
# Split into 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_encoded, test_size=0.2, random_state=42)
# Optional if you want to be sure:
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

In [19]:
from sklearn.preprocessing import StandardScaler
import pickle

# حول بياناتك من 3D إلى 2D
X_train_flat = X_train.reshape(X_train.shape[0], -1)

scaler = StandardScaler()
scaler.fit(X_train_flat)

# احفظ الـ Scaler
with open(r"D:\Graduation Project\data\hand_gesture_project\models\scaler1.pkl", "wb") as f:
    pickle.dump(scaler, f)


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import pickle

# 1. تحويل X_train و X_test من 3D إلى 2D
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [28]:
import sys
sys.path.append('D:/Graduation Project/data/hand_gesture_project')  # أو حسب المسار الكامل للمشروع عندك

In [29]:
import joblib
from src.features.extract_features import extract_features
X_train_flat = np.array([extract_features(window) for window in X_train])  # هيكون (num_windows, 36)
X_test_flat = np.array([extract_features(window) for window in X_test])  # هيكون (num_windows, 36)

In [30]:


# 2. تجهيز السكيلر وتطبيقه
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

# 3. إنشاء وتدريب النموذج
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# 4. التقييم
y_pred = rf_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print("✅ الدقة (Accuracy):", accuracy)
print("\n📊 تقرير التصنيف:\n", classification_report(y_test, y_pred))

# 5. حفظ النموذج والسكيلر
with open(r"D:\Graduation Project\data\hand_gesture_project\models\random_forest_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)

with open(r"D:\Graduation Project\data\hand_gesture_project\models\scaler1.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("✅ تم حفظ النموذج والسكيلر بنجاح")


✅ الدقة (Accuracy): 1.0

📊 تقرير التصنيف:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       220
           1       1.00      1.00      1.00       253
           2       1.00      1.00      1.00       244
           3       1.00      1.00      1.00       264
           4       1.00      1.00      1.00       218

    accuracy                           1.00      1199
   macro avg       1.00      1.00      1.00      1199
weighted avg       1.00      1.00      1.00      1199

✅ تم حفظ النموذج والسكيلر بنجاح


In [31]:
# حفظ المحول LabelEncoder
with open(r"D:\Graduation Project\data\hand_gesture_project\models\label_encoder1.pkl", "wb") as f:
    pickle.dump(label_encoder, f)